# Image Preprocessing

Importing Image Data Generator Library

In [2]:
pwd

'/home/wsuser/work'

In [3]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Rb_98YMyoN-zzee4ztUdxx8U4xexG4hGYrxQKq1GidT0',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'arrythmiaclassification-donotdelete-pr-iyy1hnmipwjeum'
object_key = 'data - Copy.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [4]:
from io import BytesIO 
import zipfile 
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist() 
for path in file_paths:
    unzip.extract(path)

In [5]:
ls

'data - Copy'/


In [6]:
from keras.preprocessing.image import ImageDataGenerator

Configure Image datagenerator class

In [7]:
#setting parameter for image data agumentationto the training data
train_datagen=ImageDataGenerator(rescale=1./225,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
#image data agumenatation to test dataset
test_datagen=ImageDataGenerator(rescale=1./225)

Applying ImageDataGenerator Functionality to train and test Dataset

In [8]:
#performing data agumentation to train the dataset
x_train=train_datagen.flow_from_directory(directory=r'/home/wsuser/work/data - Copy/train',target_size=(64,64),batch_size=32,class_mode='categorical')
#performing agumentation to test the dataset
x_test=test_datagen.flow_from_directory(directory=r'/home/wsuser/work/data - Copy/test',target_size=(64,64),batch_size=32,class_mode='categorical')


Found 15341 images belonging to 6 classes.
Found 6825 images belonging to 6 classes.


# Model Building

Importing Libraries

In [9]:

import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

from tensorflow.keras.layers import Dense,Flatten

from tensorflow.keras.layers import Conv2D,MaxPooling2D
import keras

In [10]:
model= keras.Sequential()

Adding CNN Layers

In [11]:
#adding model layer
model.add(Conv2D(32,(3,3),input_shape=(64,64,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())

Adding Dense Layers

In [12]:
model.add(Dense(32))
model.add(Dense(6,activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 32)                2

Configuring the learning process

In [13]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

Train the Model

In [14]:
model.fit_generator(generator=x_train,steps_per_epoch = len(x_train), epochs=10, validation_data=x_test,validation_steps = len(x_test))

/tmp/wsuser/ipykernel_164/53529210.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=x_train,steps_per_epoch = len(x_train), epochs=10, validation_data=x_test,validation_steps = len(x_test))


Epoch 1/10
480/480 [==============================] - 94s 196ms/step - loss: 1.1576 - accuracy: 0.5950 - val_loss: 0.8486 - val_accuracy: 0.7010
Epoch 2/10
480/480 [==============================] - 92s 191ms/step - loss: 0.3923 - accuracy: 0.8820 - val_loss: 0.5365 - val_accuracy: 0.8501
Epoch 3/10
480/480 [==============================] - 92s 191ms/step - loss: 0.2905 - accuracy: 0.9132 - val_loss: 0.4944 - val_accuracy: 0.8705
Epoch 4/10
480/480 [==============================] - 93s 193ms/step - loss: 0.2321 - accuracy: 0.9325 - val_loss: 0.3894 - val_accuracy: 0.8854
Epoch 5/10
480/480 [==============================] - 92s 192ms/step - loss: 0.2037 - accuracy: 0.9401 - val_loss: 0.4424 - val_accuracy: 0.8854
Epoch 6/10
480/480 [==============================] - 91s 190ms/step - loss: 0.1751 - accuracy: 0.9486 - val_loss: 0.3420 - val_accuracy: 0.8914
Epoch 7/10
480/480 [==============================] - 93s 193ms/step - loss: 0.1548 - accuracy: 0.9525 - val_loss: 0.3445 - val_ac

Saving the model

In [27]:
model.save('ECG.h5')

In [15]:
!pip install watson-machine-learning-client

     |████████████████████████████████| 538 kB 23.1 MB/s eta 0:00:01


In [17]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"VFECcIuGn6VBQtE-m_AjCrHtiyr7oMqmlk9Cc9xXG0_u"
}
client = APIClient(wml_credentials)

In [18]:
client

In [2]:
def guid_from_space_name(client,space_name):
    space=client.spaces.get_details()
    #print(space)
    return(next(item for item in space ['resources']if item['entity']["name"]==space_name)['metadata']['id'])

In [3]:
space_uid=guid_from_space_name(client,'arrythmia-prediction')
print("Space UID ="+space_uid)

NameError: name 'client' is not defined

In [23]:
client.set.default_space(space_uid)

'SUCCESS'

In [24]:
client.software_specifications.list(200)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1

In [25]:
software_sapce_uid=client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")

In [31]:
software_sapce_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [32]:
ls

 arrythmia-classification.tgz  'data - Copy'/   ECG.h5


In [33]:
!tar -zcvf arrythmia-classification.tgz ECG.h5

ECG.h5


In [36]:
model_details=client.repository.store_model(model='arrythmia-classification.tgz',meta_props={
    client.repository.ModelMetaNames.NAME: "arrythmia-classification",
    client.repository.ModelMetaNames.TYPE: 'tensorflow_2.7',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_sapce_uid
})

In [37]:
model_id = client.repository.get_model_id(model_details)

In [38]:
model_id

'9b2bf570-e6ab-4257-8662-215c5655becd'

In [1]:
client.repository.download(model_id,'my_model.tar.gb')

NameError: name 'client' is not defined

In [40]:
ls

 arrythmia-classification.tgz  'data - Copy'/   ECG.h5   my_model.tar.gz


Test the Model

In [11]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model=load_model('/Users/anshumanr/Documents/Externship/Project/Training/ECG.h5')
img=image.load_img("/Users/anshumanr/Documents/Externship/Project/Flask/uploads/PAC.png",target_size=(64,64))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
pred = model.predict(x)
y_pred=np.argmax(pred)
y_pred


2022-01-23 16:19:46.985099: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


2

In [12]:
index=['left Bundle Branch block','Normal','Premature Atrial Contraction','Premature Ventricular Contraction','Right Bundle Branch Block','Ventricular Fibrillation']
result = str(index[y_pred])
result

'Premature Atrial Contraction'